In [2]:
import networkx as nx
import pulp
from itertools import product
import math

In [13]:
def build_graph_from_edges(edge_list):
    G = nx.DiGraph()
    for u, v, cap, w in edge_list:
        G.add_edge(u, v, capacity=float(cap), weight=float(w))
    return G

In [14]:
def all_pairs_shortest_paths_pred_and_path(G):
    nodes = list(G.nodes())
    paths = dict(nx.all_pairs_dijkstra_path(G, weight='weight'))
    return paths

In [5]:
def compute_F_indicator(G, paths):
    """
    Compute F_l_im: a mapping (edge, (i,m)) -> 0/1
    """
    F = { (u,v): set() for u,v in G.edges() }
    nodes = list(G.nodes())
    for i in nodes:
        for m in nodes:
            if i==m: 
                continue
            path = paths.get(i, {}).get(m, None)
            if path is None:
                continue
            # path is a list of nodes: convert to edge pairs
            edges_on_path = list(zip(path[:-1], path[1:]))
            for e in edges_on_path:
                if e in F:
                    F[e].add((i,m))
    return F

In [6]:
def solve_lb_spr_lp(G, s_vector):
    """
    Build and solve LP (7) from the paper.
    Variables: k_i for each node i, and Q.
    """
    nodes = list(G.nodes())
    paths = dict(nx.all_pairs_dijkstra_path(G, weight='weight'))
    F = compute_F_indicator(G, paths)

    prob = pulp.LpProblem("LB_SPR", pulp.LpMinimize)
    # variables
    k_vars = { i: pulp.LpVariable(f"k_{i}", lowBound=0) for i in nodes }
    Q = pulp.LpVariable("Q", lowBound=0)

    # objective
    prob += Q

    # sum k_i = 1
    prob += pulp.lpSum([k_vars[i] for i in nodes]) == 1

    # edge constraints
    for (u,v) in G.edges():
        cap = G[u][v].get('capacity', 1.0)
        # sum over (i,m) in F[(u,v)]
        terms = []
        for (i,m) in F[(u,v)]:
            # k_i * s_m + k_m * s_i
            terms.append(k_vars[i] * s_vector[m] + k_vars[m] * s_vector[i])
        if terms:
            prob += (pulp.lpSum(terms) / cap) <= Q
        else:
            # no traffic uses this edge in shortest-paths -> trivially 0 <= Q (no constraint needed)
            pass

    # solve (pulp default solver, may be CBC)
    prob.solve(pulp.PULP_CBC_CMD(msg=False))

    k_res = { i: pulp.value(k_vars[i]) for i in nodes }
    Q_res = pulp.value(Q)
    return k_res, Q_res, paths, F

In [7]:
def compute_spr_worst_case(G, s_vector, paths, F):
    """
    For each directed edge l, construct bipartite flow network as described in the paper:
      x -> each i in I_l with capacity s_i
      edges (i->j) for each (i,j) in P_l with infinite capacity
      each j in J_l -> y with capacity r_j (=s_j in their model)
    Compute maxflow f_l_max, compute U_l = f_l_max / C_l
    Returns dicts f_l_max and U_l
    """
    nodes = list(G.nodes())
    fmax = {}
    U = {}
    for (u,v) in G.edges():
        pairs = F[(u,v)]
        I_l = set(i for i,m in pairs)
        J_l = set(m for i,m in pairs)
        if not pairs:
            fmax[(u,v)] = 0.0
            U[(u,v)] = 0.0
            continue

        # build flow network as DiGraph
        FG = nx.DiGraph()
        src = "SRC"
        sink = "SINK"
        # x->i
        for i in I_l:
            FG.add_edge(src, ("I",i), capacity=float(s_vector[i]))
        # j->y
        for j in J_l:
            FG.add_edge(("J",j), sink, capacity=float(s_vector[j]))
        # infinite edges between I and J for each pair
        INF = sum(s_vector.values()) * 10.0 + 1.0
        for (i,m) in pairs:
            FG.add_edge(("I",i), ("J",m), capacity=INF)
        # compute max flow
        try:
            flow_val = nx.maximum_flow_value(FG, src, sink, capacity='capacity')
        except Exception as e:
            # fallback if directed maximum flow fails for some reason
            flow_val = 0.0
        cap = G[u][v].get('capacity', 1.0)
        fmax[(u,v)] = flow_val
        U[(u,v)] = flow_val / cap if cap>0 else float('inf')
    return fmax, U

In [8]:
def compute_guaranteed_loads(s_vector, Q_value, U_dict):
    """
    LB-SPR guaranteed (paper eq. 9): s_i / Q  -> min over i gives slbr_gtd
    SPR guaranteed (paper eq. 10): s_i / U_l  -> min over i,l gives sspr_gtd
    """
    slbr_vals = { i: (s_vector[i] / Q_value if Q_value>0 else float('inf')) for i in s_vector }
    slbr_gtd = min(slbr_vals.values())

    # for SPR we need min_{i,l} s_i / U_l ; but U_l may be zero -> skip those edges (they don't constrain)
    sspr_candidates = []
    for l, U in U_dict.items():
        if U > 0:
            for i in s_vector:
                sspr_candidates.append(s_vector[i] / U)
    sspr_gtd = min(sspr_candidates) if sspr_candidates else 0.0

    return slbr_gtd, sspr_gtd, slbr_vals

In [11]:
def example_run():
    G = nx.read_gml("abvt.gml")
    print("节点数:", G.number_of_nodes())
    print("边数:", G.number_of_edges())
    G = G.to_directed()
    for node in G.nodes():
        if 'longitude' not in G.nodes[node] or 'latitude' not in G.nodes[node]:
            G.nodes[node]['Longitude'] = 0.0  # 默认经度
            G.nodes[node]['Latitude'] = 0.0   # 默认纬度
    
    for u, v in G.edges():
        # 经纬度（GraphML里有 d29=lat, d32=lon）
        lat1, lon1 = float(G.nodes[u]['Latitude']), float(G.nodes[u]['Longitude'])
        lat2, lon2 = float(G.nodes[v]['Latitude']), float(G.nodes[v]['Longitude'])
        # Haversine 距离 (近似 km)
        dist = math.sqrt((lat1-lat2)**2 + (lon1-lon2)**2)  # 简单欧几里得代替
        
        G[u][v]['weight'] = dist
        G[u][v]['capacity'] = 1.0 / (dist+1e-6)  # 容量 ~ 1/距离
    
    s_vector = {}
    for n in G.nodes():
        cap_sum = sum(G[n][nbr]['capacity'] for nbr in G.neighbors(n))
        s_vector[n] = cap_sum

    k_res, Q_res, paths, F = solve_lb_spr_lp(G, s_vector)
    fmax, U = compute_spr_worst_case(G, s_vector, paths, F)
    slbr_gtd, sspr_gtd, slbr_vals = compute_guaranteed_loads(s_vector, Q_res, U)

    print("LB-SPR results:")
    print("  Q =", Q_res)
    print("  k_i =", k_res)
    print("Guaranteed node loads (LB-SPR) per node (s_i/Q):", slbr_vals)
    print(" slbr_gtd (min over nodes) =", slbr_gtd)
    print()
    print("SPR worst-case per-edge maxflow and utilization U_l:")
    for e in fmax:
        print(f"  edge {e}: fmax={fmax[e]}, capacity={G[e[0]][e[1]]['capacity']}, U={U[e]}")
    print("sspr_gtd (min over i,l s_i/U_l) =", sspr_gtd)
    if sspr_gtd>0:
        print("Gain G = slbr_gtd / sspr_gtd =", slbr_gtd / sspr_gtd)
    else:
        print("Gain undefined (SPR guaranteed = 0)")

In [12]:
example_run()

节点数: 23
边数: 31
LB-SPR results:
  Q = 13.0
  k_i = {'Washington CDC': 0.0, 'New York': 0.0, 'Atlanta': 0.0, 'Miami': 0.37931034, 'Boston': 0.0, 'London': 0.0, 'Philadelphia': 0.0, 'Baltimore': 0.0, 'Amsterdam': 0.0, 'Frankfurt': 0.0, 'Houston': 0.0, 'None': 0.031530149, 'Paris': 0.0, 'Dallas': 0.0, 'Austin': 0.042836089, 'Seattle': 0.0, 'Portland': 0.19314694, 'Tokyo': 0.0, 'San Francisco': 0.14992631, 'Los Angeles': 0.069954664, 'Phoenix': 0.0, 'Denver': 0.13329551, 'Chicago': 0.0}
Guaranteed node loads (LB-SPR) per node (s_i/Q): {'Washington CDC': 307692.3076923077, 'New York': 384615.3846153846, 'Atlanta': 230769.23076923078, 'Miami': 153846.15384615384, 'Boston': 153846.15384615384, 'London': 307692.3076923077, 'Philadelphia': 153846.15384615384, 'Baltimore': 153846.15384615384, 'Amsterdam': 230769.23076923078, 'Frankfurt': 153846.15384615384, 'Houston': 384615.3846153846, 'None': 230769.23076923078, 'Paris': 153846.15384615384, 'Dallas': 230769.23076923078, 'Austin': 153846.1538461